In [16]:
directory_path = 'Dec23-Feb10'

In [ ]:
import os
import re

for filename in os.listdir(directory_path):
    if filename.endswith('.json') and '(' in filename and ')' in filename:
        new_name = re.sub(r'(.*)\.(.*)\((\d+)\)\.json', r'\1(\3).\2.json', filename)
        os.rename(os.path.join(directory_path, filename), os.path.join(directory_path, new_name))
        # print(new_name, 'converted')

print(f'Finished with {directory_path}')


In [18]:

from PIL import Image
from datetime import datetime
import os

def change_modified_date(file_path):
    # Check the file extension
    if file_path.endswith('.jpg') or file_path.endswith('.JPG'):
        # Open the image file
        with Image.open(file_path) as img:
            # Extract the taken date from the metadata
            taken_date = img._getexif()[36867]  # This is a common tag for taken date
    elif file_path.endswith('.mp4') or file_path.endswith('.MP4'):
        # Get the creation date of the MP4 file
        taken_date = os.path.getmtime(file_path)
    else:
        raise ValueError("Unsupported file type. Only .jpg and .mp4 files are supported.")

    # Parse the taken date to a datetime object
    if isinstance(taken_date, str):
        taken_datetime = datetime.strptime(taken_date, "%Y:%m:%d %H:%M:%S")
    else:
        taken_datetime = datetime.fromtimestamp(taken_date)

    # Convert datetime to a timestamp
    taken_timestamp = taken_datetime.timestamp()

    # Change the modified date of the file
    os.utime(file_path, (taken_timestamp, taken_timestamp))

In [71]:
def find_json_filename(directory_path, filename):
    # print(filename)

    json_file_path = f'{directory_path}/{filename}.supplemental-metadata.json'
    if os.path.exists(json_file_path):
        return json_file_path
    
    json_file_path = json_file_path.split('.')[0][:-1]+'.supplemental-metadata.json'
    if os.path.exists(json_file_path):
        return json_file_path
    
    json_file_path = json_file_path.split('.')[0]+'.supplemental-metadata.json'
    if os.path.exists(json_file_path):
        return json_file_path
    
    if '(' in filename and ')' in filename and '.' in filename:
        json_file_path = f"{directory_path}/{filename.split('(')[0]}.{filename.split('.')[-1]}({filename.split('(')[1].split(')')[0]}).supplemental-metadata.json"
        if os.path.exists(json_file_path):
            return json_file_path
        
    json_file_path = f'{directory_path}/{filename}.suppl.json'
    if os.path.exists(json_file_path):
        return json_file_path
    
    json_file_path = f'{directory_path}/{filename}.supplemen.json'
    if os.path.exists(json_file_path):
        return json_file_path

    else:
        raise FileNotFoundError(f'JSON file not found for {json_file_path}')
        # change_modified_date(f'{directory_path}/{filename}')
        # return None


In [72]:
import json
import os
import time
from PIL import Image
from datetime import datetime

from tqdm.notebook import tqdm

for filename in tqdm(os.listdir(directory_path), desc=f"Processing Files in {directory_path}"):
    # print(f'working on {filename}')
    if not filename.endswith('.json'):
        # Load the JSON data
        if filename.endswith('.MP4'):
            # Try to find the corresponding .JPG or .HEIC file
            mp4name = filename
            jpgname = filename.replace('.MP4', '.JPG')
            heicname = filename.replace('.MP4', '.HEIC')
            if os.path.exists(os.path.join(directory_path, jpgname)):
                jsonname = jpgname
            elif os.path.exists(os.path.join(directory_path, heicname)):
                jsonname = heicname
            elif os.path.exists(os.path.join(directory_path, heicname)):
                jsonname = mp4name
            else:
                change_modified_date(f'{directory_path}/{filename}')
                continue
                # raise FileNotFoundError(f'No corresponding .JPG or .HEIC or .MP4 file found for {filename}')
            
            json_file_path = os.path.join(directory_path, f'{jsonname}.supplemental-metadata.json')
            
            with open(f'{directory_path}/{jsonname}.json', 'r') as json_file:
                data = json.load(json_file)
            # Extract the photoTakenTime timestamp
            photo_taken_timestamp = int(data['photoTakenTime']['timestamp'])

            # Convert the timestamp to the format required for setting the file's modification time
            formatted_time = time.gmtime(photo_taken_timestamp)
            modification_time = time.mktime(formatted_time)

            # Set the modified time of file
            new_file_path = f'{directory_path}/{filename}'
            os.utime(new_file_path, (modification_time, modification_time))

            # print(f"{new_file_path} Modified date updated successfully!")

        else:
            json_file_path = find_json_filename(directory_path, filename)
            # json_file_path = f'{directory_path}/{filename}.supplemental-metadata.json'
            # print(json_file_path)
            # if not os.path.exists(json_file_path):
            #     # Try to find the JSON file without the '0.jpg' part
            #     # json_file_path = os.path.join(f'{json_file_path.replace("0.jpg", "").replace("0.heic", "")}')
            #     json_file_path = json_file_path.split('.')[0][:-1]+'.json'
            #     if not os.path.exists(json_file_path):
            #         json_file_name = f"{json_file_path.split('(')[0]}.{json_file_path.split('.')[-1]}({json_file_path.split('(')[1].split(')')[0]}).supplemental-metadata.json"
            #         if not os.path.exists(json_file_path):
            #             # raise FileNotFoundError(f'JSON file not found for {json_file_path}')
            #             change_modified_date(f'{directory_path}/{filename}')

            #         continue
            
            with open(json_file_path, 'r') as json_file:
            
            #with open(f'{directory_path}/{filename}.json', 'r') as json_file:
                data = json.load(json_file)

            # Extract the photoTakenTime timestamp
            photo_taken_timestamp = int(data['photoTakenTime']['timestamp'])

            # Convert the timestamp to the format required for setting the file's modification time
            formatted_time = time.gmtime(photo_taken_timestamp)
            modification_time = time.mktime(formatted_time)

            # Set the modified time of file
            new_file_path = f'{directory_path}/{filename}'
            os.utime(new_file_path, (modification_time, modification_time))

            # print(f"{new_file_path} Modified date updated successfully!")
print(f'Finished with {directory_path}')




Processing Files in Dec23-Feb10:   0%|          | 0/2431 [00:00<?, ?it/s]

Finished with Dec23-Feb10
